# **Air quality - GIOS data analysis**
### Advanced Databases
### Automatic Control and Robotic, Cyber-physical Systems
#### **Daniel Jurkowski (407200) & Adam Pękala (405380) & Rafał Maciasz (405560) & Jakub Sacha (000000)**

## **1. Import required packages** 

In [1]:
import psycopg2
import requests
import pandas as pd
import matplotlib.pyplot as plt
import schedule
import time

## **2. Connect to database**

In [2]:
db_name = 'gios_data'
db_user = 'postgres'
db_password = 'kochamadb'
db_host = 'localhost'
db_port = '5000'

conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

## **3. Scrap and prepare data from chosen API.**

### **3.1. GET /pjp-api/rest/station/findAll** - List of all stations with cities details

In [ ]:
stations_data = pd.DataFrame.from_dict(requests.get("https://api.gios.gov.pl/pjp-api/rest/station/findAll").json())
stations_data.rename(columns={"id": "stationId"}, inplace=True)

cities_data = pd.DataFrame.from_dict(list(stations_data["city"]))
cities_data.rename(columns={"id": "cityId"}, inplace=True)
cities_data = pd.concat([cities_data, pd.DataFrame.from_dict(list(cities_data["commune"]))], axis=1)
cities_data.drop(columns=["commune"], inplace=True)

stations_data = pd.concat([stations_data, cities_data], axis=1)
stations_data.drop(columns=["city"], inplace=True)

cities_data = cities_data.drop_duplicates(subset=["cityId"], keep="last").reset_index(drop=True)

In [5]:
stations_data.head()

,stationId,stationName,gegrLat,gegrLon,addressStreet,cityId,name,communeName,districtName,provinceName
0,114,"Wrocław, ul. Bartnicza",51.115933,17.141125,ul. Bartnicza,1064,Wrocław,Wrocław,Wrocław,DOLNOŚLĄSKIE
1,117,"Wrocław, wyb. Conrada-Korzeniowskiego",51.129378,17.029250,ul. Wyb. J.Conrada-Korzeniowskiego 18,1064,Wrocław,Wrocław,Wrocław,DOLNOŚLĄSKIE
2,129,"Wrocław, al. Wiśniowa",51.086225,17.012689,al. Wiśniowa/ul. Powst. Śląskich,1064,Wrocław,Wrocław,Wrocław,DOLNOŚLĄSKIE
3,52,"Legnica, al. Rzeczypospolitej",51.204503,16.180513,al. Rzeczypospolitej 10/12,453,Legnica,Legnica,Legnica,DOLNOŚLĄSKIE
4,109,"Wałbrzych, ul. Wysockiego",50.768729,16.269677,ul. Wysockiego 11,998,Wałbrzych,Wałbrzych,Wałbrzych,DOLNOŚLĄSKIE


In [6]:
cities_data.head()

,cityId,name,communeName,districtName,provinceName
0,1064,Wrocław,Wrocław,Wrocław,DOLNOŚLĄSKIE
1,453,Legnica,Legnica,Legnica,DOLNOŚLĄSKIE
2,998,Wałbrzych,Wałbrzych,Wałbrzych,DOLNOŚLĄSKIE
3,142,Czerniawa,Świeradów-Zdrój,lubański,DOLNOŚLĄSKIE
4,198,Dzierżoniów,Dzierżoniów,dzierżoniowski,DOLNOŚLĄSKIE


### **3.2. GET /pjp-api/rest/station/sensors/{station_id}** - List of all sensors available in each station 

In [9]:
sensors_data = pd.DataFrame.from_dict(sum([requests.get(f"https://api.gios.gov.pl/pjp-api/rest/station/sensors/{station_id}").json() for station_id in stations_data["stationId"]], []))  # Use sum to flatten 2d array
sensors_data.rename(columns={"id": "sensorId"}, inplace=True)

params_data = pd.DataFrame.from_dict(list(sensors_data["param"]))
params_data.rename(columns={"idParam": "paramId"}, inplace=True)

sensors_data = pd.concat([sensors_data, params_data], axis=1)
sensors_data.drop(columns=["param"], inplace=True)

params_data = params_data.drop_duplicates(subset=["paramId"], keep="last").reset_index(drop=True)

In [10]:
sensors_data.head()

,sensorId,stationId,paramName,paramFormula,paramCode,paramId
0,642,114,dwutlenek azotu,NO2,NO2,6
1,644,114,ozon,O3,O3,5
2,672,117,dwutlenek siarki,SO2,SO2,1
3,658,117,benzen,C6H6,C6H6,10
4,660,117,tlenek węgla,CO,CO,8


In [11]:
params_data.head()

,paramName,paramFormula,paramCode,paramId
0,ozon,O3,O3,5
1,dwutlenek siarki,SO2,SO2,1
2,tlenek węgla,CO,CO,8
3,benzen,C6H6,C6H6,10
4,dwutlenek azotu,NO2,NO2,6


### **3.3. GET /pjp-api/v1/rest/data/getData/{idSensor}** - Network service making available measurement data based on the given measurement site ID, from the last full hour up to the third day back. The measurement results are made available in local time, i.e. in summer in CEST time (UTC+ 02:00, Central European Summer Time), in winter in CET time (UTC+ 01:00, Central European Winter Time).

In [18]:
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS actual_measurements (
    station_id VARCHAR(255),
    date_time TIMESTAMP,
    value DECIMAL
);
''')
conn.commit()

def insert_data_to_db(conn, data):
    cur = conn.cursor()
    if data['Lista danych pomiarowych']:
        for measurement in data['Lista danych pomiarowych']:
            cur.execute('''
                INSERT INTO actual_measurements (station_id, date_time, value)
                VALUES (%s, %s, %s);
            ''', (
                measurement['Kod stanowiska'],
                measurement['Data'],
                measurement['Wartość']
            ))
        conn.commit()

def fetch_data_from_api(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        return None
    
all_ids = stations_data["stationId"]

for id in all_ids:
    api_url = f'https://api.gios.gov.pl/pjp-api/v1/rest/data/getData/{id}'

    data = fetch_data_from_api(api_url)

    if data:
        insert_data_to_db(conn, data)
    

### **3.4. GET - /pjp-api/v1/rest/archivalData/getDataBySensor/{idSensor}** - A web service that makes available archival measurement data based on the given measurement site ID. One-hour measurement results are always made available in CET time (UTC+ 01:00, Central European Winter Time).

We have selected a query retrieving data on pollution from all sensors for the last 30 days.

approx time ~ 11min

In [ ]:
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS archive_measurements(
    station_id VARCHAR(255),
    date_time TIMESTAMP,
    value DECIMAL
);
''')
conn.commit()

def insert_data_to_db(conn, data):
    cur = conn.cursor()
    if data['Lista archiwalnych wyników pomiarów']:
        for measurement in data['Lista archiwalnych wyników pomiarów']:
            cur.execute('''
                INSERT INTO archive_measurements (station_name, station_id, date_time, value)
                VALUES (%s, %s, %s, %s);
            ''', (
                measurement['Nazwa stacji'],
                measurement['Kod stanowiska'],
                measurement['Data'],
                measurement['Wartość']
            ))
        conn.commit()

def fetch_data_from_api(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(response.text)
        return None

for id in all_ids:
    api_url = f'https://api.gios.gov.pl/pjp-api/v1/rest/archivalData/getDataBySensor/{id}?dayNumber=30&size=500'
    
    data = fetch_data_from_api(api_url)

    if data:
        insert_data_to_db(conn, data)


### **3.5. GET - /pjp-api/v1/rest/aqindex/getIndex/{stationId}** - Network service providing an air quality index based on the given measuring station identifier.


In order to realise real-time data download to the database, the script is placed in a docker container so that it works independently of the machine on which the rest of the code is executed. 

service_code.py

docker build -t service_air_index_live .

docker run -d --name service_container service_air_index_live

docker start service_container

In [ ]:
#### TEST ####
cur.execute("SELECT * FROM archive_measurements;")

rows = cur.fetchall()

cur.close()
conn.close()

df = pd.DataFrame(rows, columns=['station_id', 'date_time', 'value'])

print(df.head())

plt.figure(figsize=(10,6))
plt.plot(df['date_time'], df['value'])
plt.xlabel('Date Time')
plt.ylabel('Value')
plt.title('Measurements over time')
plt.show()

## **4. Prepare tables for measurement infrastructure data**

Data that describe measurement infrastructure such stations, sensors, etc. may bring us a lot of information about our system arrangement. Having that we are able to visualize air conditions more precisely in specific locations using maps. 

### **4.1. Create required tables**

In [15]:

cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS cities (
    city_id INTEGER PRIMARY KEY,
    name VARCHAR(255),
    province VARCHAR(255)
);
''')
conn.commit()


cur.execute('''
CREATE TABLE IF NOT EXISTS stations (
    station_id INTEGER PRIMARY KEY,
    city_id INTEGER,
    name VARCHAR(255),
    address_street VARCHAR(255),
    latitude FLOAT,
    longitude FLOAT,
    FOREIGN KEY (city_id)
        REFERENCES cities (city_id)
        ON UPDATE CASCADE ON DELETE CASCADE
);
''')
conn.commit()


cur.execute('''
CREATE TABLE IF NOT EXISTS params (
    param_id INTEGER PRIMARY KEY,
    name VARCHAR(255),
    formula VARCHAR(255),
    code VARCHAR(255)
);
''')
conn.commit()


cur.execute('''
CREATE TABLE IF NOT EXISTS sensors (
    sensor_id INTEGER PRIMARY KEY,
    station_id INTEGER,
    param_id INTEGER,
    FOREIGN KEY (station_id)
        REFERENCES stations (station_id)
        ON UPDATE CASCADE ON DELETE CASCADE,
    FOREIGN KEY (param_id)
        REFERENCES params (param_id)
        ON UPDATE CASCADE ON DELETE CASCADE
);
''')
conn.commit()

### **4.2. Fill tables with scrapped data**

In [22]:
cur = conn.cursor()

for _, record in cities_data.iterrows():
    cur.execute(f'''
        INSERT INTO cities (city_id, name, province)
        VALUES ({record["cityId"]}, '{record["name"]}', '{record["provinceName"]}');
    ''')
conn.commit()

for _, record in stations_data.iterrows():
    cur.execute(f'''
        INSERT INTO stations (station_id, city_id, name, address_street, latitude, longitude)
        VALUES ({record["stationId"]}, {record["cityId"]}, '{record["stationName"]}', '{record["addressStreet"]}', '{record["gegrLat"]}', '{record["gegrLon"]}');
    ''')
conn.commit()

for _, record in params_data.iterrows():
    cur.execute(f'''
        INSERT INTO params (param_id, name, formula, code)
        VALUES ({record["paramId"]}, '{record["paramName"]}', '{record["paramFormula"]}', '{record["paramCode"]}');
    ''')
conn.commit()

for _, record in sensors_data.iterrows():
    cur.execute(f'''
        INSERT INTO sensors (sensor_id, station_id, param_id)
        VALUES ({record["sensorId"]}, {record["stationId"]}, {record["paramId"]});
    ''')
conn.commit()